#**07-2 심층 신경망**

In [ ]:
# 실행마다 동일한 결과를 얻기 위해 케라스에 랜덤 시드를 사용하고 텐서플로 연산을 결정적으로 만듭니다.
import tensorflow as tf

tf.keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()

In [ ]:
# 케라스 API를 사용해서 패션 MNIST 데이터셋 불러오기

from tensorflow import keras

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
from sklearn.model_selection import train_test_split

# 픽셀값 0~255에서 0~1 사이로 변경
train_scaled = train_input / 255.0

# 2차원배열을 1차원 배열로 변경
train_scaled = train_scaled.reshape(-1, 28*28)

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

<img src= "https://drive.google.com/uc?export=view&id=1QQYPoqWbNfUrcoorD9bsmqI_2a05wUR5" widrh=300>

입력층과 출력층 사이에 밀집층이 추가됨
- **입력층과 출력층 사이에 있는 모든 층을 은닉층(hidden layer)이라고 부름**

은닉층에는 주황색 원으로 활성화 함수 표시
- **활성화 함수는 신경망 층의 선형 방정식의 계산 값에 적용하는 함수**
- 소프트맥스 함수도 활성화 함수 임
- 출력층에 적용하는 활성화 함수는 종류가 제한되어 있음
 - 이진 분류의 경우 시그모이드, 다중 분류는 소프트맥스
 - 은닉층의 활성화 함수는 비교적 자유로움
  - 대표적으로 시그뫼드 함수와 볼 렐루(LeRU) 함수 등

은닉층에 활성화 함수를 적용하는 이유?

a x 4 + 2 = b <br>
b x 3 - 5 = c <br>
첫 번째 식에서 계산된 b가 두 번째 식에서 c를 계산하기 위해 사용됨
하지만 두 번째 식에 첫 번째 식을 대입하면 <br>
a x 12 + 1 = c <br>
이렇게 b가 사라지고 하나로 합쳐짐

<br>

신경망에서도 역시 은닉층에서 선형적인 산술 계산만 수행하면 수행 역할이 없는 셈이 되므로 선형 계산을 적당하게 비선형적으로 비틀어 주어야 다음 층의 계산과 단순히 합쳐지지 않고 나름의 역할을 할 수 있음

a x 4 = b <br>
log(b) = k <br>
k x 3 - 5 = c

시그모이드 함수는 뉴런의 출력 z 값을 0과 1 사이로 압축

In [ ]:
# 시그모이드 활성화 함수를 사용한 은닉층과 소프트 맥스 함수를 사용한 출력층을 케라스의 Dense 클래스로 생성
# 케라스에서 신경망의 첫 번째 층은 input_shape 매개변수로 입력의 크기를 지정해 줘야만 함

dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
dense2 = keras.layers.Dense(10, activation='softmax')

dense1
- 은닉층이고 100개의 뉴런을 가진 밀집 층
- 활성화 함수를 'sigmoid'로 지정
- 입력의 크기를 (784,)로 지정
- 은닉층의 뉴런 개수를 정하는 데에는 기준 X
 - 적어도 출력층의 뉴런 보다는 많아야 함

dense 2
- 10개의 클래스를 분류함로 10개의 뉴런
- 활성화 함수는 소프트 맥스 함수

# 심층 신경망 만들기


In [ ]:
# dense1과 dense2 객체를 Sequential 클래스에 추가해 심층 신경망(deep neaural) 생성

model = keras.Sequential([dense1, dense2])

<img src= "https://drive.google.com/uc?export=view&id=16OjAA0IjARGH8AAW8sYTaCBMGB7EMTyD" widrh=300>

**Sequential 클래스의 객체를 만들 때 여러 개의 층을 추가하려면 이처럼 dense1과 dense2를 리스트로 만들어 전달**
- 주의할 것은 출력층을 가장 마지막에 두어야 함
- 이 리스트는 가장 처음 등장하는 은닉층에서 마지막 출력층의 순서로 나열해야 함

**인공 신경망의 강력한 성능은 이렇게 층을 추가하여 입력 데이터에 대해 연속적인 학습을 진행하는 능력에서 나옴**
- 2개 이상의 층을 추가할 수도 있음

In [ ]:
# 케라스는 모델의 summary() 메서드를 호출하면 층에 대한 유용한 정보를 얻을 수 있음

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


1. 맨 첫 줄에 모델의 이름
2. 그 다음 이 모델에 들어 있는 층이 순서대로 나열
- 맨 처음 추가한 은닉층에서 출력층의 순서대로
- 층마다 층 이름, 클래스, 출력 크기, 모델 파라미터 개수가 출력됨
 - 층을 만들 때 name 매개변수로 이름 지정 가능, 지정x 시 자동으로 'dense'

 - 출력 크기가 (None, 100)임
   - 첫 번째 차원은 샘플의 개수, 샘플 개수가 정의되지 않기 때문에 None
   - **케라스 모델의 fit() 메서드에 훈련 데이터를 주입하면 이 데이터를 한 번에 모두 사용하지 않고 잘게 나누어 여러 번에 걸쳐 경사 하강법 단계 수행(미니 배치 경사 하강법)**
   - 케라스의 기본 미니 배치는 32개, 이 값은 fit() 메서드에서 batch_size 매개변수로 바꿀 수 있음
   - 따라서 샘플 개수를 고정하지 않고 어떤 배치 크기에도 유연하게 대응할 수 있도록 None으로 지정
   - **이렇게 신경망 층에 입력되거나 출력되는 배열의 첫 번째 차원을 배치 차원이라고 부름**
   <br>

   -  두 번째 100은 은닉층의 뉴런 개수를 100개로 두어 100개의 출력이 나옴, 즉 샘플마다 784개의 픽셀값이 은닉층을 통과하면서 100개의 특성으로 압축
  
  - 마지막으로 모델 파라미터 개수 출력
   - 이 층은 Dense 층이므로 입력 픽셀 784개와 100개의 모든 조합에 대한 가중치가 있음
   - 뉴런마다 1개의 절편이 있음
    <img src= "https://drive.google.com/uc?export=view&id=1uYsoqYSSJWbf83j58Yuqu4KiNtw1psUr" widrh=300>

   - 두 번째 층의 출력 크기는 (None, 10)임
    - 배치 차원은 동일하게 None이고 출력 뉴런 개수가 10개이기 때문
    - 이 층의 모델 파라미터 개수는 100개의 은닉층 뉴런가 10개의 출력층 뉴런이 모두 연결되고 출력층의 뉴런마다 하나의 절편이 존재하므로 1010개
   <img src= "https://drive.google.com/uc?export=view&id=1SXxatHDQltiT-NFGvdMEPSUSTNqw4AAB" widrh=300>

- summary() 메서드의 마지막에는 총 모델 파라미터 개수와 훈련되는 파라미터 개수가 동일하게 79510개로 나옴
  - 은닉층과 출력층의 파라미터 개수를 합친 것
  - 그 아래 훈련되지 않은 파라미터는 0으로 나옴
   - 간혹 경사 하강법으로 훈련되지 않는 파라미터를 가진 층이 있는데, 그 개수가 나옴

# 층을 추가하는 다른 방법

**Sequential 클래스의 생성자 안에서 바로 Dense 클래스의 객체를 만드는 경우가 많음**

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
    keras.layers.Dense(10, activation='softmax', name='output')
], name='패션 MNIST 모델')

특징

- 이렇게 작업시 추가되는 층을 한 눈에 쉽게 알아볼 수 있음
- 모델의 이름과 달리 층의 이름은 반드시 영문이어야만 함
- 여러 모델과 많은 층을 사용할 경우 name 매개변수를 사용하면 구분이 쉬움
- 아주 많은 층을 추가하려면 Sequential 클래스 생성자가 매우 길어지는 단점
- 조건에 따라 층을 추가할 수도 없음

In [ ]:
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Sequential 클래스에서 층을 추가할 때 가장 널리 사용하는 방법은 모델의 <font color ="red">add() 메서드</font>
- Sequential 클래스의 객체를 만들고 이 객체의 add() 메서드를 호출하여 층을 추가
- 한 눈에 추가되는 층을 볼 수 있고 프로그램 실행 시 동적으로 층을 선택하여 추가할 수 있음

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# 5번의 에포크동안 모델 훈련

model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs = 5)

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5710 - accuracy: 0.8064
Epoch 2/5
1500/1500 [==============================] - 5s 4ms/step - loss: 0.4132 - accuracy: 0.8509
Epoch 3/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3776 - accuracy: 0.8646
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3530 - accuracy: 0.8732
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3344 - accuracy: 0.8782


추가된 층이 선응을 향상 시킴
- 인공 신경망에 몇개의 층을 추가하더라도 compile()과 fit()의 사용법은 동일

# 렐루 함수

렐루함수(ReLU)
- 시그모이드 함수는 오른쪽과 왼쪽 끝으로 갈 수록 그래프가 누워있기 때문에 올바른 출력을 만드는데 신속한 대응이 어려움
- 특히 층이 많을수록 학습을 더 어렵게 함
- 이를 개선하기 위해 나온 다른 종류의 활성화 함수가 렐루함수
- 렐루 함수는 입력이 양수일 경우 마치 활성화 함수가 없는 것 처럼 그냥 입력을 통과 시키고 음수일 경우는 0으로 만듬
   <img src= "https://drive.google.com/uc?export=view&id=1fc_Vj-HrVH7vjUcQPWndeubmL1J31uSP" widrh=300>



렐루 함수는 max(z, 0)와 같이 쓸 수 있음
- 이 함수는 z가 0보다 크면 z를 출력하고 z가 0보다 작으면 0을 출력함

**렐루 함수는 특히 이미지 처리에서 좋은 성능을 낸다고 알려짐**

패션 MNIST 데이터는 28x28 크기 이기 때문에 인공 신경망에 주입하기 위해 넘파이 배열의 reshape() 메서드를 사용해 1차원으로 펼쳤는데, 케라스에서는 이를 위한 Flatten 층을 제공함
- Flatten 클래스는 배치 차원을 제외하고 나머지 입력 차원을 모두 일렬로 펼치는 역할만함
- 입력에 곱해지는 가중치나 절편 X (인공 신경망의 성능에 기여 X)
- 하지만 Flatten 클래스를 층처럼 입력층과 은닉층 사이에 추가하기 때문에 이를 층이라고 부름
- Flatten 층은 입력층 바로 뒤에 추가함

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

- 첫 번째 Dense 층에 있던 input_shape 매개변수를 Flatten 층으로 옮김
- 첫 번째 Dense 층의 활성화 함수를 'relu'로 변경
- 그러나 Flatten 층은 학습하는 층이 아니므로 이 신경망을 깊이가 3인 신경망이라고 부르지 않음

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


첫 번째 등장하는 Flatten 클래스에 포함된 모델 파라미터는 0

**케라스의 Flatten 층을 신경망 모델에 추가하면 입력값의 차원을 짐작할 수 있는 것이 또 하나의 장점**
- 앞의 출력에서 784개의 입력이 첫 번째 은닉층에 전달되는 것을 확인 가능
 - 이전 모델에서는 쉽게 확인 불가
 - 입력 데이터에 대한 천처리 과정을 가능한 모델에 포함 식키는 것이 케라스 API의 철학

In [ ]:
# 훈련 데이터 준비 (이 절의 서두에 있던 코드와 동일하지만 reshape() 적용 X)

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

# 이전 코드에선 이 코드 이후 reshape 적용
train_scaled = train_input / 255.0

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 7s 4ms/step - loss: 0.5291 - accuracy: 0.8113
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3924 - accuracy: 0.8580
Epoch 3/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3525 - accuracy: 0.8719
Epoch 4/5
1500/1500 [==============================] - 7s 4ms/step - loss: 0.3318 - accuracy: 0.8816
Epoch 5/5
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3154 - accuracy: 0.8855


In [ ]:
# 검증 세트에서의 성능 확인

model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3865 - accuracy: 0.8668


[0.3864709138870239, 0.8668333292007446]

시그모이드 함수를 사용했을 때, 은닉층을 추가하지 않은 경우보다 성능 향상

# 옵티마이저

신경망에는 특히 하이퍼파라미터가 많이 존재
- 지금까지 다룬 하이퍼파라미터는 추가할 은닉층의 개수, 뉴런 개수, 활성화 함수, 층의 종류, 배치 사이즈 매개변수, 에포크 매개변수 등

**추가할 은닉층의 개수, 은닉층의 뉴런 개수, 활성화 함수, 층의 종류, fit() 메서드의 배치 사이즈 매개변수, 에포크 매개 변수, 케라스의 옵티마이저, RMSprop의 학습률 모두 하이퍼파라미터**
- **케라스는 다양한 종류의 경사 하강법 알고리즘을 제공하는데 이들을 옵티마이저(optimizer)라고 함**
- 가장 기본적인 옵티마이저는 확률적 경사하강법인 SGD, 이름은 SGD이지만 1개의 샘플을 뽑아서 훈련하지 않고 앞서 언급한 것처럼 미니 배치를 사용함



In [ ]:
# SGD 옵티마이저를 사용하려면 compile() 메서드의 optimizer 매개변수를 'sgd'로 지정
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

In [ ]:
# 이 옵티마이저는 tensorflow.keras.optimizers 패키지 아래 SGD 클래스로 구현됨
# 'sgd' 문자열은 이 클래스의 기본 설정 매개변수로 생성한 객체와 동일, 즉 아래 코드는 위의 코드와 정확히 동일

sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')

원래 sgd = keras.optimizers.SGD() 처럼 SGD 클래스 객체를 만들어 사용해야 하는데 번거로움을 피하고자 'sgd'라고 지정하면 자동으로 SGD 클래스 객체를 만들어줌

In [ ]:
# SGD 클래스의 학습률 기본값이 0.01일때, 이를 바꾸고 싶으면 learning_rate 매개변수에 지정

sgd = keras.optimizers.SGD(learning_rate=0.1)

<img src= "https://drive.google.com/uc?export=view&id=1M1Adz5_HvscsMfD7puQBvSXJUjdB1gZA" widrh=300>

**SGD 이외에 다양한 옵티마이저**
- 기본 경사 하강법 옵티마이저는 모두 SGD 클래스에서 제공
 - SGD 클래스의 momentum 매개변수의 기본값은 0
 - 이를 0보다 큰 값으로 지정하면 마치 이전의 그레이디언트를 가속도처럼 사용하는 모멘텀 최적화(momentum optimization)를 사용, 보통 momentum 매개변수는 0.9이상을 지정
- SGD 클래스의 nesterov 매개변수를 기본값 False에서 True로 바꾸면 네스테로프 모멘텀 최적화 (nesterov momentum optimization)(또는 네스테로프 가속 경사)를 사용

In [ ]:
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)

**네스테로프 모멘텀은 모멘텀 최적화를 2번 반복하여 구현**
- 대부분 기본 확률적 경사 하강법보다 나은 성능 제공

모델이 최적점에 가까이 갈수록 학습률을 낮출 수 있음
- 이렇게 하면 안정적으로 최적점에 수렴할 가능선이 높음
- **이런 학습률을 적응적 학습률(adaptive learning rate)이라고 함**
- 이런 방식들은 학습률 매개변수를 튜닝하는 수고를 덜어줌

**적응적 학습률을 사용하는 대표적 옵티마이저는 Adagrad와 RMSprop**
- 각각 compile() 메서드의 optimizer 매개변수에 'adagrad'와 'rmsprop'으로 지정
- optimizer 매개변수의 기본값은 'rmsprop'
- 이 두 옵티마이저의 매개변수를 바꾸고 싶다면 SGD와 같이 Adagrad와 RMSprop 클래스 객체를 만들어 사용하면 됨


In [ ]:
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [ ]:
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics='accuracy')

**모멘텀 최적화와 RMSprop의 장점을 접목한 것이 Adam**
- Adam 클래스도 keras.optimizers 패키지 아래에 있음
- learning_rate는 기본값으로 역시 0.001 사용

In [ ]:
# 모델 다시 만듬

model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [ ]:
# optimizer를 adam으로

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 7s 4ms/step - loss: 0.5262 - accuracy: 0.8154
Epoch 2/5
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3940 - accuracy: 0.8589
Epoch 3/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3546 - accuracy: 0.8702
Epoch 4/5
1500/1500 [==============================] - 8s 6ms/step - loss: 0.3284 - accuracy: 0.8794
Epoch 5/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3058 - accuracy: 0.8872


In [ ]:
# 검증 세트 성능 확인

model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3509 - accuracy: 0.8751


[0.3508652448654175, 0.875083327293396]